In [1]:
# 基础库
import numpy as np
import os
import pandas as pd
import random
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import cv2
# 图像.dcm文件读取
import pydicom

In [2]:
def  trans(x):
    mean = np.mean(x)
    std = np.std(x)
    x = transforms.ToTensor()(x)
    x = transforms.Normalize(mean,std)(x)
    return x

In [3]:
def dicom_data(path):
    data_array = []
    for j in os.listdir(path):
    # 遍历每个患者内所有影像数据
        dcm_path = path + '/' + j
        slices = [pydicom.read_file(dcm_path + '/' + s) for s in os.listdir(dcm_path)]  # 读取每一个影像数据内部所有影像
        for s in slices:
            # 不改变分辨率的情况下重塑张量至 (335, 335)
            resized_image = cv2.resize(s.pixel_array, (335, 335), interpolation=cv2.INTER_AREA)
            data_array.append(resized_image)  # 更新数据列表，将得到的影像添加上去    
    return np.array(data_array)

In [ ]:
def clinical_data(sample_name):
    # 读取CSV文件
    df = pd.read_csv('data.csv')
    
    # 找到样本名称对应的行
    row = df[df.iloc[:,0] == sample_name]
    
    # 返回除了第一列之外的其他数值
    return row.iloc[:,1:]

In [4]:
data_path1 = './dataset/first_test/0/'
datas = dict()
for i in os.listdir(data_path1):
    datas[i] = dict()
    data_array = CMR_data(data_path1+i)
    datas[i]['CMR'] = data_array
    datas[i]['label'] = 0

In [5]:
data_path2 = './dataset/first_test/1/'
for i in os.listdir(data_path2):
    datas[i] = dict()
    data_array = CMR_data(data_path2+i)
    datas[i]['CMR'] = data_array
    datas[i]['label'] = 1

In [6]:
features1 = []
labels = []
for j in datas:
    arr = datas[j]['CMR']
    arr  = arr.astype(np.float64)
    arr = trans(arr).numpy()
    features1.append(arr)
    labels.append(datas[j]['label'])
features1 = np.array(features1)
labels = np.array(labels)

In [7]:
features1 = features1.reshape(features1.shape[0],1,features1.shape[1],features1.shape[2],features1.shape[3])
print(features1.shape)
print(labels.shape)

(56, 1, 335, 27, 335)
(56,)


In [8]:
save_path1 = './dataloader/feature1.npy'
save_path2 = './dataloader/label.npy'

In [9]:
np.save(save_path1,features1)
np.save(save_path2,labels)